In [1]:
import os
os.chdir('/mnt/jw01-aruk-home01/projects/psa_functional_genomics/RA_challenge/RA2_alpine_lads')

In [2]:
import pandas as pd
import numpy as np
import os
from utils.config import Config
test_config = Config()
from dataset.train_dataset import train_dataset

In [3]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from tensorflow import keras 

In [4]:
dataset = train_dataset(test_config)

In [5]:
hands_dataset,feet_dataset,hands_dataset_val,feet_dataset_val = dataset.initialize_pipeline()

In [6]:
feet_dataset

<PrefetchDataset shapes: ((None, 150, 105, 1), (None, 13)), types: (tf.float32, tf.int64)>

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt

In [8]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(test_config.feet_height,test_config.feet_width ,1)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(13)
])

In [9]:
model.compile(loss='mean_squared_error', optimizer='adam')

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 105, 16)      160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 52, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 52, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 26, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 37, 26, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 18, 13, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 14976)             0

In [16]:
history = model.fit(
    feet_dataset,validation_data=feet_dataset_val,epochs=10, steps_per_epoch=150,
    validation_steps=3
)

Train for 150 steps, validate for 3 steps
Epoch 1/10
150/150 [==============================] - 1s 4ms/step - loss: 0.0583 - val_loss: 9.4595
Epoch 2/10
150/150 [==============================] - 1s 4ms/step - loss: 0.0191 - val_loss: 9.5873
Epoch 3/10
150/150 [==============================] - 1s 4ms/step - loss: 0.0118 - val_loss: 9.6825
Epoch 4/10
150/150 [==============================] - 1s 4ms/step - loss: 0.0517 - val_loss: 9.4881
Epoch 5/10
150/150 [==============================] - 1s 4ms/step - loss: 0.0529 - val_loss: 6.8243
Epoch 6/10
150/150 [==============================] - 1s 4ms/step - loss: 0.0859 - val_loss: 6.7033
Epoch 7/10
150/150 [==============================] - 1s 4ms/step - loss: 0.0679 - val_loss: 9.6110
Epoch 8/10
150/150 [==============================] - 1s 4ms/step - loss: 0.0454 - val_loss: 9.9349
Epoch 9/10
150/150 [==============================] - 1s 4ms/step - loss: 0.0404 - val_loss: 9.5849
Epoch 10/10
150/150 [==============================] - 1s 

In [ ]:
model.predict(feet_dataset,steps = 10)

In [ ]:
for image, label in feet_dataset.take(1):
    plt.imshow(np.repeat(np.array(image[0]),3,axis=2),cmap="Greys")
    print(label[0])
    print(model.predict(np.array(image[0]).reshape(1,150,105,1)))
    print("Image shape: ", image.numpy().shape)
    print("Label: ", label.numpy())

In [22]:
base_model = tf.keras.applications.ResNet50V2(input_shape=(test_config.feet_height,test_config.feet_width ,1),
                                               include_top=False,weights=None
                                               )

In [23]:
base_model.summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 105, 1) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 156, 111, 1)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 75, 53, 64)   3200        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 77, 55, 64)   0           conv1_conv[0][0]                 
_________________________________________________________________________________________

In [26]:
model1 = tf.keras.Sequential([
    base_model,
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(13)
])

In [27]:
model1.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Model)           (None, 5, 4, 2048)        23558528  
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 2, 2, 2048)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               4194816   
_________________________________________________________________
dense_5 (Dense)              (None, 13)                6669      
Total params: 27,760,013
Trainable params: 27,714,573
Non-trainable params: 45,440
_________________________________________________________________


In [31]:
model1.compile(loss='mean_squared_error', optimizer='adam')

In [37]:
history = model1.fit(
    feet_dataset,validation_data=feet_dataset_val,epochs=100, steps_per_epoch=150,
    validation_steps=3
)

Train for 150 steps, validate for 3 steps
Epoch 1/100
150/150 [==============================] - 5s 33ms/step - loss: 3.9274 - val_loss: 8.3993
Epoch 2/100
150/150 [==============================] - 5s 33ms/step - loss: 4.1527 - val_loss: 8.6648
Epoch 3/100
150/150 [==============================] - 5s 33ms/step - loss: 3.9279 - val_loss: 8.6735
Epoch 4/100
150/150 [==============================] - 5s 33ms/step - loss: 4.1519 - val_loss: 8.6600
Epoch 5/100
150/150 [==============================] - 5s 33ms/step - loss: 3.9972 - val_loss: 8.6656
Epoch 6/100
150/150 [==============================] - 5s 33ms/step - loss: 4.0829 - val_loss: 8.6610
Epoch 7/100
150/150 [==============================] - 5s 33ms/step - loss: 4.0866 - val_loss: 7.9380
Epoch 8/100
150/150 [==============================] - 5s 33ms/step - loss: 3.9851 - val_loss: 8.6667
Epoch 9/100
150/150 [==============================] - 5s 33ms/step - loss: 3.9593 - val_loss: 8.6717
Epoch 10/100
150/150 [==================

KeyboardInterrupt: 